<a href="https://colab.research.google.com/github/Pullsu/Uczenie-maszynowe-w-finansach/blob/Use-Case-3/Use_case_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Strategia inwestycyjna (decyzje wejścia i wyjścia) dla spółki  Microsoft (MSFT), test w okresie od 01.01.2024 - 06.05.2024

In [2]:
!pip install yfinance backtesting scikit-learn ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=b471e55a2f8530c997a084e9c0477725b663ec9d6fc574d25b678b1d2547607b
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29411 sha256=30229dfd327a21fe822b66ed21a156b43a018039bb96eb81921c6f7d289bc7be
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built backtesting ta


In [9]:
!pip install ta

In [154]:
import yfinance as yf
import pandas as pd
import numpy as np
import ta
from backtesting import Backtest, Strategy
from backtesting.lib import SignalStrategy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import accuracy_score
from ta.momentum import RSIIndicator
from ta.trend import SMAIndicator, ADXIndicator
from ta.volatility import BollingerBands, AverageTrueRange

In [155]:
data = yf.download('MSFT', start='2024-01-01', end='2024-05-06')
data['Return'] = data['Close'].pct_change()
data.dropna(inplace=True)

[*********************100%%**********************]  1 of 1 completed


In [156]:
data['SMA'] = ta.trend.sma_indicator(data['Close'], window=14)
data['SMA_2w'] = ta.trend.sma_indicator(data['Close'], window=2*14)
data['RSI'] = ta.momentum.rsi(data['Close'], window=14)
data['ADX'] = ta.trend.adx(data['High'], data['Low'], data['Close'], window=14)
bb = BollingerBands(data['Close'])
data['BB_upper'] = bb.bollinger_hband()
data['BB_middle'] = bb.bollinger_mavg()
data['BB_lower'] = bb.bollinger_lband()
data['ATR'] = ta.volatility.average_true_range(data['High'], data['Low'], data['Close'], window=14)
data['DI_pos'] = ta.trend.adx_pos(data['High'], data['Low'], data['Close'], window=14)
data['DI_neg'] = ta.trend.adx_neg(data['High'], data['Low'], data['Close'], window=14)
data.dropna(inplace=True)

In [157]:
features = ['SMA', 'SMA_2w', 'RSI', 'ADX', 'BB_upper', 'ATR', 'DI_pos', 'DI_neg']
X = data[features]
y = np.where(data['Return'].shift(-1) > 0, 1, np.where(data['Return'].shift(-1) < 0, -1, 0))

In [165]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Define parameter grid for GridSearch
param_grid_rf = {
    'n_estimators': [10, 50, 100],
    'max_depth': [20, 30],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 4, 6],
}

In [182]:
# Train the model using GridSearchCV
model_rf = RandomForestClassifier(random_state=42)
tscv = TimeSeriesSplit(n_splits=10)
grid_search_rf = GridSearchCV(estimator=model_rf, param_grid=param_grid_rf, cv=tscv)
grid_search_rf.fit(X_train, y_train)


# Najlepszy model z GridSearch
best_model_rf = grid_search_rf.best_estimator_

# Przewidywanie na zbiorze testowym
data['Test_Predictions'] = np.nan
data.loc[X_test.index, 'Test_Predictions'] = best_model_rf.predict(X_test)

# Wyznaczenie sygnałów kupna i sprzedaży
training_data_len = len(X_train)
data['Signal'] = np.nan
data.loc[data.index[training_data_len:], 'Signal'] = np.where(
    data['Test_Predictions'][training_data_len:] > data['Open'][training_data_len:], 1, -1
)
data['Position'] = data['Signal'].shift()

In [183]:
class MLStrategy(SignalStrategy):
    def init(self):
        super().init()
        self.signal = self.I(lambda: data['Position'].to_numpy(), name='signal')

    def next(self):
        current_signal = self.signal[-1]
        if current_signal == 1:
            self.buy(sl=self.data.Close[-1] * 0.95, tp=self.data.Close[-1] * 1.05)
        elif current_signal == -1:
            self.sell(sl=self.data.Close[-1] * 1.05, tp=self.data.Close[-1] * 0.95)

# Przygotowanie danych dla backtestu
data.index = pd.to_datetime(data.index)

# Uruchomienie backtestu
bt = Backtest(data, MLStrategy, cash=10000, commission=.002, exclusive_orders=True)
output = bt.run()
bt.plot()

# Print results
print(output)

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2024-02-12 00:00:00
End                       2024-05-03 00:00:00
Duration                     81 days 00:00:00
Exposure Time [%]                   15.517241
Equity Final [$]                 10303.916276
Equity Peak [$]                  10843.263118
Return [%]                           3.039163
Buy & Hold Return [%]               -2.070993
Return (Ann.) [%]                   13.891901
Volatility (Ann.) [%]               16.357906
Sharpe Ratio                         0.849247
Sortino Ratio                        1.976693
Calmar Ratio                         2.792888
Max. Drawdown [%]                   -4.974027
Avg. Drawdown [%]                   -3.260726
Max. Drawdown Duration        3 days 00:00:00
Avg. Drawdown Duration        3 days 00:00:00
# Trades                                    9
Win Rate [%]                        55.555556
Best Trade [%]                       3.599072
Worst Trade [%]                      -1.51909
Avg. Trade [%]                    

## Podsumowanie:
* Czas trwania backtestu (Duration): 81 dni.

* Equity final = 10,303.92 **Kapitał końcowy jest wyższy niż początkowy kapitał 10,000, co oznacza zysk**

* Equity Peak = 10,843.26

* **strategia wygenerowała 3,04% zysku na zakończenie okresu backtestu**

* Strategia Buy & Hold przyniosłaby stratę w tym samym okresie (-2,07%)

* Przewidywany roczny zwrot strategii - 13,89%

* Sharpe Ratio: 0.85

Sharpe Ratio mierzy średni zwrot ponad stopę wolną od ryzyka na jednostkę zmienności. Wartość powyżej 0.5 jest ogólnie uznawana za dobrą.

* Sortino Ratio: 1.98

Sortino Ratio jest podobne do Sharpe Ratio, ale koncentruje się tylko na zmienności związanej ze spadkami cen. Wartość powyżej 1 jest uznawana za dobrą.

* Calmar Ratio: 2.79

Calmar Ratio mierzy roczny zwrot w stosunku do maksymalnego obsunięcia. Wartość powyżej 1 jest uznawana za dobrą.

* Całkowita liczba transakcji wykonanych przez strategię - 9

### Strategia inwestycyjna działa dobrze, generując zyski przy relatywnie niskiej zmienności. Wskaźniki jakości, takie jak Sharpe Ratio, Sortino Ratio i Calmar Ratio, sugerują, że strategia jest zyskowna i stabilna. Należy jednak pamiętać, że liczba transakcji jest stosunkowo niska (9), co może wpływać na dokładność oceny strategii w dłuższym okresie.

### Warto rozważyć dalsze testowanie strategii na większej ilości danych oraz różnorodnych warunkach rynkowych, aby upewnić się co do jej skuteczności.